# Pinecone Retrieval Pipeline with Reranking for RAG

### Modular Pinecone Retrieval Pipeline for RAG

This notebook demonstrates an end-to-end retrieval pipeline for a Retrieval-Augmented Generation (RAG) system using Pinecone as the vector database and GoogleGenerativeAIEmbeddings for embedding creation.

The pipeline supports modular retrieval, optional Maximal Marginal Relevance (MMR) reranking, and is easily extendable for additional features such as BM25 reranking, LLM-based generation, or custom filtering.

**Contents:**
- Setup and Prerequisites
- Retriever Function
- MMR Reranking Function
- Pipeline Wrapper
- Usage Example


In [ ]:
import os
import glob
from uuid import uuid4
from dotenv import load_dotenv

from pinecone import Pinecone, ServerlessSpec

from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker

load_dotenv()

## Set up your Environment and API keys

In [ ]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")
os.environ['PINECONE_API_KEY']=os.getenv("PINECONE_API_KEY")

## **Data Loading and Semantic Chunking Pipeline**

This code block performs two key preprocessing steps to prepare your PDFs for retrieval-augmented generation (RAG):

---

### **Step 1: Load All PDFs from a Directory**

- **Reads all PDF files** from the `./data/` folder using a file pattern (`*.pdf`).
- **Loads each PDF** using `PyPDFLoader`, which extracts the content (typically one Document per page).
- **Aggregates all page Documents** from all PDFs into a single list, `all_docs`.
- **Prints the total number of loaded page documents**—helpful for verifying that your data covers the expected number of pages (should be >200 for a large-scale use case).

---

### **Step 2: Semantic Chunking of the Documents**

- **Initializes your embedding model** (`GoogleGenerativeAIEmbeddings`), which will be used to embed text and assist semantic chunking.
- **Creates a `SemanticChunker`** with:
    - The embedding model
    - A breakpoint method (`standard_deviation`) for smart, context-aware chunking (not just fixed-length)
    - `min_chunk_size` to ensure chunks are not too small.
- **Uses the chunker to split each document’s text** into semantically meaningful chunks, resulting in `semantic_chunks`.
    - Each chunk is likely to contain a paragraph or section, preserving context for better retrieval and LLM answers.
- **Prints the total number of semantic chunks** generated for tracking and diagnostics.

---

### **Step 3: Embed All Semantic Chunks**

- **Extracts the raw text from each semantic chunk** into the list `texts`.
- **Embeds each chunk** using the embedding model’s `.embed_documents()` method, resulting in a list of embedding vectors, one per chunk.
- **Prints the length of one embedding vector** to confirm the embedding size (e.g., 768 for Google embeddings).

---

**In summary:**  
This code loads all your PDFs, splits them into high-quality semantic chunks using a neural chunker, and generates dense vector embeddings for each chunk. These embeddings and chunks are now ready to be added to your vector database for fast semantic search and retrieval.


In [ ]:
# Step 1: Load all PDFs from directory
pdf_dir = "./data/"
pdf_files = glob.glob(os.path.join(pdf_dir, "*.pdf"))

all_docs = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    all_docs.extend(loader.load())

print(f"Loaded {len(all_docs)} page documents (should cover >200 pages).")

# Step 2: Semantic Chunking
embedding_model = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
chunker = SemanticChunker(embedding_model, breakpoint_threshold_type="standard_deviation", min_chunk_size=100)
semantic_chunks = chunker.create_documents([doc.page_content for doc in all_docs])

print(f"Total semantic chunks: {len(semantic_chunks)}")

texts = [chunk.page_content for chunk in semantic_chunks]
embeddings = embedding_model.embed_documents(texts)
print(len(embeddings[0]))

## **Index Creation and Data Ingestion with Pinecone**

This code block sets up the Pinecone vector database, creates an index for storing your embeddings, and upserts (uploads) your embedded document chunks into the index for future retrieval.

---

### **1. Initialize Pinecone Client**

- Instantiates a Pinecone client object, which provides methods to interact with the Pinecone service.

---

### **2. Create (or Connect to) the Vector Index**

- **Defines the index name** (e.g., `"agenticbatch2-assignment"`).
- **Checks if the index already exists**; if not, creates it with:
    - `dimension=768`: The size of your embedding vectors (must match your model).
    - `metric="cosine"`: Uses cosine similarity for measuring vector similarity.
    - `spec=ServerlessSpec(cloud="aws", region="us-east-1")`: Specifies the cloud provider and region for your index.

---

### **3. Load the Index and Initialize the Vector Store**

- Loads the index for further operations.
- Initializes a `PineconeVectorStore` using the loaded index and your embedding model.  
  This makes it easy to add documents using high-level LangChain methods if needed.

---

### **4. Prepare and Upsert Embedding Data**

- **Creates a list of items** to insert:
    - Each item is a tuple: `(unique_id, embedding_vector, metadata)`, where
        - `unique_id`: A new UUID for each chunk,
        - `embedding_vector`: The vector for the chunk,
        - `metadata`: Typically stores the original text or provenance info.
- **Uploads the items to Pinecone** in batches (batch size = 10), which is more efficient and avoids timeouts for large datasets.
- **Upsert** is used so you can add new data or update existing data seamlessly.

---

**Result:**  
After this block runs, your semantic document chunks are stored in the Pinecone index as vectors with metadata. You can now perform fast, scalable vecto


In [ ]:
pc=Pinecone()

index_name="agenticbatch2-assignment"

if not pc.has_index(index_name):
    pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws",region="us-east-1")    
)
    
#loading the index
index=pc.Index(index_name)
vector_store=PineconeVectorStore(index=index,embedding=embedding_model)

# Each item: (id, vector, metadata)
items = [
    (str(uuid4()), emb, {"text": text}) for emb, text in zip(embeddings, texts)
]

batch_size = 10
for i in range(0, len(items), batch_size):
    index.upsert(vectors=items[i:i+batch_size])

## Optional: If you want to use different inexing techniques, you can create multiple indexes in Pinecode and test their performances

In [ ]:
# INDEX_DIM = 768  # Must match your embedding size

# # Flat index
# pc.create_index(
#     name="rag-flat",
#     dimension=INDEX_DIM,
#     metric="cosine",
#     spec=ServerlessSpec(cloud="gcp", region="us-east1"),
#     pod_type="s1",     # or appropriate for your plan
#     index_type="POD",  # POD is Pinecone's Flat index
# )

# # HNSW index
# pc.create_index(
#     name="rag-hnsw",
#     dimension=INDEX_DIM,
#     metric="cosine",
#     spec=ServerlessSpec(cloud="gcp", region="us-east1"),
#     pod_type="s1",
#     index_type="HNSW"
# )

# # IVF_PQ index (Pinecone's version of IVF)
# pc.create_index(
#     name="rag-ivfpq",
#     dimension=INDEX_DIM,
#     metric="cosine",
#     spec=ServerlessSpec(cloud="gcp", region="us-east1"),
#     pod_type="s1",
#     index_type="IVF_PQ"
# )

## **Retriever Function for Semantic Search**

This function enables semantic search over your Pinecone index using your embedding model. It takes a user query and retrieves the most relevant document chunks based on vector similarity.

---

### **How It Works**

- **Input:**  
  - `query`: The user’s search string.
  - `top_k`: The number of top results to retrieve (default: 10).
  - `threshold`: Minimum similarity score for a match to be considered relevant.

- **Process:**
  1. **Embed the query:**  
     The user’s query is converted into a dense vector using the embedding model (`embed_query`).
  2. **Query the Pinecone index:**  
     The query vector is compared against all vectors in the index to find the `top_k` most similar ones.
     - `include_metadata=True`: Returns each match’s associated metadata (such as the original text).
     - `include_values=True`: Returns each match’s embedding vector (useful for reranking).
     - `filter=None`: No additional filtering is applied, but you can use this to restrict search by metadata (e.g., source or document type).
  3. **Score threshold filtering:**  
     Only results with a similarity `score` greater than or equal to `threshold` are kept.
  4. **Returns:**  
     - `matches`: The filtered list of top matches (each including metadata and embedding).
     - `query_emb`: The embedding vector for the user’s query (useful for reranking or analysis).

---

**Usage:**  
Call this function with a query string to get the most semantically similar chunks from your vector database for downstream processing, reranking, or as LLM context.


In [ ]:
def retrieve(query, top_k=10, threshold=0.7):
    query_emb = embedding_model.embed_query(query)
    # Query Pinecone
    result = index.query(
        vector=query_emb,
        top_k=top_k,
        include_metadata=True,
        filter=None,  # Optionally filter on metadata
        include_values=True
    )
    # Filter by score threshold
    matches = [m for m in result['matches'] if m['score'] >= threshold]
    return matches, query_emb


## **Benchmarking Retrieval Speed and Inspecting Results**

This code block demonstrates how to **measure the response time** of your retrieval pipeline and to **inspect the top retrieved results** for a specific query.

---

### **How It Works**

- **Defines a sample query** (`test_query`) relevant to your domain.
- **Records the current time** before running the retrieval function.
- **Calls the `retrieve` function** with the test query to obtain relevant document matches from Pinecone.
- **Calculates the retrieval time** by subtracting the start time from the current time after retrieval completes.
- **Prints the retrieval latency** (in seconds), giving you a sense of system performance.
- **Iterates over the returned matches** to print:
    - The similarity score of each result (how well it matches the query).
    - The first 200 characters of the retrieved text for a quick preview.

---

**Usage:**  
This benchmarking step is useful for evaluating the speed and quality of your search pipeline, tuning hyperparameters (like `top_k` or `threshold`), or comparing different index types (Flat, HNSW, IVF_PQ).


In [ ]:
import time

test_query = "How Radiology Report Generation is achieved using transfer learning?"

start = time.time()
matches, _ = retrieve(test_query)
elapsed = time.time() - start

print(f"Retrieval time: {elapsed:.4f}s")
for m in matches:
    print(f"Score: {m['score']:.2f}, Text: {m['metadata']['text'][:200]}...")


## **Reranking Search Results with Maximal Marginal Relevance (MMR)**

This code block introduces **MMR (Maximal Marginal Relevance) reranking** to further improve the quality of retrieved documents.  
MMR helps ensure the final selected context is both highly relevant to the query and non-redundant (diverse), reducing repetition and surfacing varied supporting evidence.

---

### **How It Works**

1. **MMR Function Definition**
    - `mmr()` takes the query embedding, document embeddings, texts, and reranking parameters.
    - It calculates the cosine similarity between the query and each document.
    - It then iteratively selects the next document that maximizes both relevance to the query and diversity from already selected documents.
    - `lambda_param` (typically between 0.5 and 0.8) controls the trade-off between relevance (higher) and diversity (lower).

2. **Applying the MMR Pipeline**
    - Calls `retrieve()` with a user query to get candidate matches (including their embeddings and texts).
    - Extracts all document embeddings and corresponding texts from the matches.
    - Passes these to `mmr()` to select the top `k` (e.g., 5) reranked chunks.
    - Prints the top reranked document for inspection.

---

**Usage:**  
MMR reranking is a best practice in RAG systems to reduce answer redundancy, increase answer quality, and help LLMs synthesize responses from a broader context set.



In [ ]:
# Reranking

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def mmr(query_emb, doc_embs, texts, top_k=5, lambda_param=0.7):
    doc_embs = np.array(doc_embs)
    query_emb = np.array(query_emb).reshape(1, -1)
    sim = cosine_similarity(doc_embs, query_emb).flatten()
    selected, candidates = [], list(range(len(texts)))
    while len(selected) < top_k and candidates:
        if not selected:
            idx = int(np.argmax(sim))
        else:
            sim_to_query = sim[candidates]
            sim_to_selected = np.max(cosine_similarity(doc_embs[candidates], doc_embs[selected]), axis=1)
            mmr_scores = lambda_param * sim_to_query - (1 - lambda_param) * sim_to_selected
            idx = candidates[int(np.argmax(mmr_scores))]
        selected.append(idx)
        candidates.remove(idx)
    return [texts[i] for i in selected]


# 1. Get matches and query embedding
matches, query_emb = retrieve("How Radiology Report Generation is achieved using transfer learning?", top_k=10)

# 2. Prepare for MMR
doc_embs = [m['values'] for m in matches]
texts = [m['metadata']['text'] for m in matches]

# 3. Rerank
reranked_mmr = mmr(query_emb, doc_embs, texts, top_k=5)
print("Top reranked doc:", reranked_mmr[0][:300])

## **LLM Answer Generation with Prompt Engineering**

This code block takes the final, reranked retrieved context and uses it as input to a Large Language Model (LLM) to generate a concise, context-grounded answer.

---

### **How It Works**

1. **Prompt Template Construction**
    - Defines a clear, instruction-based prompt for the LLM, specifying its role as a research assistant.
    - Uses placeholders `{context}` and `{question}` to dynamically insert the most relevant document snippets and the user's question.

2. **Prepare Context and Question**
    - Selects the top 3 reranked document chunks (`reranked_mmr[:3]`), joining them into a single string as context.
    - Uses the test query as the question.

3. **LLM Initialization and Invocation**
    - Initializes the LLM (Google Gemini) using `ChatGoogleGenerativeAI`.
    - Fills the prompt template with the selected context and question.
    - Sends the filled prompt to the LLM for answer generation.
    - Prints the generated answer.

---

**Usage:**  
This step combines retrieved, high-quality context with the user’s question, then leverages the LLM’s reasoning and synthesis abilities to produce a focused, well-grounded answer—closing the loop of the RAG pipeline.



In [ ]:
prompt_template = """You are a research assistant. Use the context below to answer the question concisely.
Context:
{context}
Question: {question}
Answer:"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

context = "\n\n".join(reranked_mmr[:3])
question = test_query

llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
final_prompt = prompt.format(context=context, question=question)
llm_response = llm.invoke(final_prompt)
print("LLM output:\n", llm_response)


# LangChain Retrieval Pipeline

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.7}  # Hyperparameter: tune as needed
)

In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
        Question: {question} 
        Context: {context} 
        Answer:""",
    input_variables=['context', 'question']
)

In [ ]:
# ---- 3. DOC FORMATTER ----

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# ---- 4. RAG CHAIN ----

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
# ---- 5. INTERACTIVE QUERY FUNCTION ----

def run_rag_pipeline(question):
    print(f"\n=== User Query ===\n{question}\n")

    # Retrieve context documents (for transparency and debug)
    retrieved_docs = retriever.invoke(question)
    print(f"Retrieved {len(retrieved_docs)} context documents:")
    for i, doc in enumerate(retrieved_docs, 1):
        preview = doc.page_content[:300] + ("..." if len(doc.page_content) > 200 else "")
        print(f"{i}. {preview}  [source: {doc.metadata.get('source','')}]")

    print("\nGenerating LLM Answer...\n")
    answer = rag_chain.invoke(question)
    print("=== LLM Answer ===\n", answer)
    return answer

In [ ]:
run_rag_pipeline("How Radiology Report Generation is achieved using transfer learning?")